In [258]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from math import log, e
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [303]:
#VERSIONE CON USO DEI MOTIF,DISCORD O ENTRAMBI E CON RIMOZIONE DEI CANDIDATI GIA SCELTI

In [260]:
def retrieve_all(Ts): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [261]:
#riceve la lista di coppie dei motifs per ogni record(Ts), e resittuisce lista di valori singoli

def candidateFilter(CandidatesList): 
    counterNumberMotif=0
    counterNumberDiscord=0
    l2=np.array([])
    for i in range (len(CandidatesList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(CandidatesList['Motif'].iloc[i])
        numDiscord=len(CandidatesList['Discord'].iloc[i])
        counterNumberDiscord+=numDiscord
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=CandidatesList['Motif'].iloc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista
            counterNumberMotif+=1
            
    
        CandidatesList['Motif'].iloc[i]=l2
        l2=np.array([]) #svuoto array
    
    return CandidatesList,counterNumberMotif,counterNumberDiscord

In [262]:
def buildCandidatesUsedList(CandidatesList):
    CandidatesUsedList=pd.DataFrame(columns=['Used'],index=range(0,numberOfMotif+numberOfDiscord))
    boolList=[False] * (numberOfMotif+numberOfDiscord)
    CandidatesUsedList['Used']= boolList
    return CandidatesUsedList
                                    

In [263]:
#carico dataset in dataFrame
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
df = pd.DataFrame(dataset[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
#trasformo da stringa a numero il campo target 
le = LabelEncoder()
num_classes = le.fit_transform(df['target'])
df['target']=num_classes
df['TsIndex']=np.arange(len(df))

print(df)
Ts = np.array(df.iloc[0][0:-2].values)
print(Ts)

        att1      att2      att3      att4      att5      att6      att7  \
0  -0.464276 -0.555048 -0.842843 -0.865895 -0.936396 -0.817270 -0.263612   
1  -0.896972 -0.685686 -1.351382 -1.458667 -1.165346 -1.403929 -1.821800   
2  -0.464696 -0.567739 -0.032023 -0.635046 -0.602826 -0.266856 -0.267061   
3  -0.187190 -0.620808 -0.815661 -0.521398 -0.790423 -0.967517 -1.487006   
4  -1.136017 -1.319195 -1.844624 -0.788692 -0.251715 -1.487603 -0.668764   
5  -0.568709 -0.888118 -0.047977 -0.038036 -0.483068 -1.094689 -0.953453   
6  -0.965839 -1.644583 -1.805935 -1.499658 -0.920088 -1.115706 -0.650195   
7  -0.264302 -0.290925 -0.598567 -0.518863 -0.488594 -0.407091 -1.071489   
8  -0.803716 -0.048161 -1.299278 -1.072280 -1.004018 -0.511452 -1.173215   
9  -1.186422 -0.893197 -0.977737 -0.634109 -1.384820 -1.271926 -1.212171   
10 -0.653308 -1.295185 -0.457384 -1.092036 -1.682346 -0.461811 -0.099362   
11 -0.541876 -1.016107 -1.079902 -1.185665 -0.792452 -0.754593 -0.057041   
12 -1.945327

In [299]:
window_size=5
#diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
diz={'Motif':[],'Discord':[]}

#CALCOLO MOTIF E DISCORD E LI INSERISCO NEL DIZIONARIO
for i in range(30):
    Ts = np.array(df.iloc[i][0:-2].values)
    mp,mot,motif_dist,dis = retrieve_all(Ts)
    diz['Motif'].insert(i, mot)
    diz['Discord'].insert(i, dis)

#GENERO DFRAME DA DIZIONARIO

CandidatesList = pd.DataFrame(diz)
CandidatesList,numberOfMotif,numberOfDiscord=candidateFilter(CandidatesList)
CandidatesUsedList=buildCandidatesUsedList(CandidatesList)

print('Candidati estratti')
#print(MotifsList['Motif'])
print(CandidatesList)
print(numberOfMotif,numberOfDiscord)
print(CandidatesUsedList)

Candidati estratti
            Motif    Discord
0     [9.0, 26.0]   [61, 27]
1   [105.0, 54.0]   [8, 119]
2    [17.0, 40.0]    [31, 1]
3    [48.0, 12.0]   [23, 86]
4          [27.0]   [38, 62]
5    [45.0, 13.0]  [119, 26]
6     [7.0, 37.0]   [98, 76]
7     [2.0, 24.0]    [36, 8]
8     [1.0, 29.0]   [23, 32]
9     [50.0, 0.0]   [38, 59]
10   [11.0, 17.0]   [80, 23]
11    [31.0, 7.0]  [114, 54]
12         [53.0]   [61, 10]
13    [79.0, 6.0]  [105, 38]
14         [59.0]    [98, 8]
15    [2.0, 45.0]     [1, 8]
16    [28.0, 3.0]    [5, 88]
17   [14.0, 35.0]  [118, 85]
18    [13.0, 3.0]   [111, 8]
19    [19.0, 6.0]   [92, 35]
20   [19.0, 39.0]   [26, 70]
21         [26.0]   [13, 35]
22   [18.0, 22.0]   [59, 25]
23   [27.0, 42.0]  [32, 104]
24    [8.0, 28.0]    [4, 95]
25         [16.0]   [87, 20]
26   [67.0, 59.0]   [33, 83]
27   [16.0, 39.0]   [75, 99]
28   [11.0, 41.0]   [47, 39]
29    [6.0, 15.0]  [117, 51]
55 60
      Used
0    False
1    False
2    False
3    False
4    False
..     ...

In [265]:
#per ogni Ts calcolo Dprofile con ogni candidato e inserisco la distanza minima con candidato i-esimo nella colonna i-esima
def computeSubSeqDistance(dataset,CandidatesList):
    
    #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    numberOfCandidates=0
    for i in range(len(CandidateList)):
            numberOfCandidates+=len(CandidatesList['Motif'].loc[i])
            numberOfCandidates+=len(CandidatesList['Discord'].loc[i])
    columnsList=np.arange(numberOfCandidates)
    columnsList2=list()
    lastAttribute=['TsIndex','class']
    prefix='cand'
    for i in columnsList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(dataset)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(dataset)):
        #acquisisco la Ts
        TsToCompare = np.array(dataset.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for j in range(len(CandidatesList)):
            numMotif=len(CandidatesList['Motif'].iloc[j])
            numDiscord=len(CandidatesList['Discord'].iloc[j])
            for k in range(numMotif):
                l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
            for k in range(numDiscord):
                l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
        
   # print(counter)    
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                
                

In [266]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset, num_classes_unique):
    numPattern=len(dataset)
    if(numPattern>0):
        #print('numPattern: '+str(numPattern))
        relativeFrequencies=list()
        counter=0 #num pattern di ogni classe nel dataset
        #ottengo frequenze relative
        for i in (num_classes_unique):
            for j in range(numPattern):
                if(dataset.iloc[j]['class']==i):
                    counter+=1
            relativeFrequencies.append(counter/numPattern)
            counter=0
        #calcolo entropia dataset
        #print('relativeFrequencies: '+str(relativeFrequencies))
        entropy=0
        for k in (relativeFrequencies):
            if(k!=0):
                entropy+=(k*log(k,2))
        return -entropy
    else:
        return 0

In [267]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli
def computeGain(entropyParent,LenDatasetParent,Dleft,Dright,num_classes_unique):
    entropyLeft=computeEntropy(Dleft,num_classes_unique)
    entropyRight=computeEntropy(Dright,num_classes_unique)
    gain=entropyParent
    summation=( ((len(Dleft)/LenDatasetParent)*entropyLeft) +  ((len(Dright)/LenDatasetParent)*entropyRight) )
    #print('entropyParent: '+str(entropyParent))
    #print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [268]:
#SPLIT SLAVE
#effettua lo split del dataset sul attributo e valore fornito
def split(dataset,attribute,value): 
    columnsList=dataset.columns.values
    dizLeft=pd.DataFrame(columns=columnsList)
    dizRight=pd.DataFrame(columns=columnsList)
    for i in range(len(dataset)):
        if dataset.iloc[i][attribute] < value:
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
    return dizLeft, dizRight

In [300]:
def getBestIndexAttribute(vecMutualInfo,candidatesGroup):
    print(len(vecMutualInfo))
    print(numberOfMotif)
    print(numberOfDiscord)
    print(numberOfMotif+numberOfDiscord)
    if(candidatesGroup==0):
        candidatesIndex=range(numberOfMotif)
    elif(candidatesGroup==1):
        candidatesIndex=range(numberOfMotif,numberOfMotif+numberOfDiscord)
    else:
        candidatesIndex=range(numberOfMotif+numberOfDiscord)
        
    MutualInfoDf=pd.DataFrame(columns=['candidatesIndex','gain'],index=range(len(candidatesIndex)))
    MutualInfoDf['gain']=vecMutualInfo
    MutualInfoDf['candidatesIndex']=candidatesIndex
    
    MutualInfoDf=MutualInfoDf.sort_values(by='gain',ascending = False)
    print('MUTUALDF')
    print(len(vecMutualInfo))
    
    bestIndexAttribute=-1
    i=0
    
    while(bestIndexAttribute==-1 and i<len(vecMutualInfo)):    
        print(i)
        attributeToVerify=int(MutualInfoDf.iloc[i]['candidatesIndex'])
        print(attributeToVerify)
        print(CandidatesUsedList.iloc[attributeToVerify]['Used'])
        if(CandidatesUsedList.iloc[attributeToVerify]['Used']==False):
            bestIndexAttribute=attributeToVerify
            CandidatesUsedList.iloc[attributeToVerify]=True
            print('BEST')
            print(bestIndexAttribute)
        else:
            i+=1
    
    return bestIndexAttribute
            
    
        
        

In [282]:
#SPLIT INTERMEDIO
#dato il dataset, cerca il miglior attributo e relativo valore (optimal split point) su cui splittare
# restituiendo il dataset splittato e i valori trovati
def findBestAttributeValue(dataset,candidatesGroup):
    #cerca e restituisce attributo migliore su cui splittaree relativo valore ottimale (optimal split point)
    #CANDIDATE GROUP permette di scegliere se usare come candidati 0=motifs 1=discord 2=entrambi
    bestGain=0
    actualGain=0
    bestvalueForSplit=0
    y = dataset['class'].values
    y=y.astype('int')
    
    #trovo best Attribute
    numAttributes=len(dataset.columns.values)
    numAttributes-=2 #tolgo i due attributi TsIndex e class dal Dframe
    datasetForMutual=pd.DataFrame()
    
    #preparo il Dframe da passare a mutual_info_classif, settando se scegliere tra motifs/discord/entrambi
    if(candidatesGroup==0): #solo motifs
        datasetForMutual=dataset.iloc[:,np.r_[:numberOfMotif]]
    elif(candidatesGroup==1):
        datasetForMutual=dataset.iloc[:,np.r_[numberOfMotif:numberOfMotif+numberOfDiscord]]
    else:
        datasetForMutual=dataset.iloc[:,np.r_[:numAttributes]]
        
    vecMutualInfo=mutual_info_classif(datasetForMutual, y, discrete_features=False)
    indexBestAttribute=getBestIndexAttribute(vecMutualInfo,candidatesGroup)
    
    
    num_classes_unique=np.unique(num_classes) 
    entropyParent=computeEntropy(dataset, num_classes_unique)
    DleftChosen=pd.DataFrame()
    DrightChosen=pd.DataFrame()
    #splitto su tale attributo e trovo il valore ottimale testando su tutte le distanze calcolate tre le Ts e tale candidato 
    for i in range (len(dataset)):
        testValue=dataset.iloc[i][indexBestAttribute]
        if(testValue>0): #distanze non-negative, split inutile su tale valore
            Dleft,Dright=split(dataset,indexBestAttribute,testValue)
            actualGain=computeGain(entropyParent,len(dataset),Dleft,Dright,num_classes_unique)
            if(actualGain > bestGain):
                bestGain=actualGain
                bestvalueForSplit=testValue
                DleftChosen=Dleft
                DrightChosen=Dright

    
    return indexBestAttribute,bestvalueForSplit,DleftChosen,DrightChosen

In [283]:
#SPLIT MASTER
# funzione ricorsiva che implementa la creazione dell'albero di classificazione
# memorizza in ogni nodo: attributo, valore attributo su cui splitto, entropia nodo, num pattern
# memorizza in ogni foglia: entropia nodo, num pattern, classe nodo

# VERSIONE CHE NON RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 

def buildTree(actualNode,dataset,maxDepth, minSize, depth,num_classes_unique):
    #caso base: num pattern < soglia minima || profondità massima raggiunta => genero foglia con media delle classi
    #DATASET HA SEMPRE ALMENO UN PATTERN
    if(len(dataset)<minSize or depth>=maxDepth ):
        average = sum(dataset['class'].values) / len(dataset['class'].values)
        classValue = round(average)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset, num_classes_unique)
        
        nodeInfo=list()
        nodeInfo.append(classValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
    
        actualNode.data=nodeInfo
        actualNode.value=-1
        actualNode.left=None
        actualNode.right=None
        #print(dataset['class'])
        return 
    #caso in cui si può splittare
    else:
        indexChosenAttribute,attributeValue,Dleft,Dright=findBestAttributeValue(dataset,0)
        
        numPattern=len(dataset)
        entropy=computeEntropy(dataset, num_classes_unique)
        
        #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
        nodeInfo=list()
        nodeInfo.append(attributeValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
        actualNode.data=nodeInfo
        actualNode.value=(indexChosenAttribute)
        
        #se possibile richiamo ricorsivamente sul nodo dx e sx figlio
        print('profondità: '+str(depth))
        print('len dataset: '+str(len(dataset)))
        print('valore: '+str(attributeValue))
        if(len(Dleft)>0):
            #Dleft=Dleft.drop(columns=[str(indexChosenAttribute)])
            actualNode.left=Node(indexChosenAttribute)
            print('scendo a sx splittando su: '+str(indexChosenAttribute))
            buildTree(actualNode.left,Dleft,maxDepth, minSize, depth+1,num_classes_unique)
        
        if(len(Dright)>0):
            #Dright=Dright.drop(columns=[str(indexChosenAttribute)])
            actualNode.right=Node(indexChosenAttribute)
            print('scendo a dx splittando su: '+str(indexChosenAttribute))
            buildTree(actualNode.right,Dright,maxDepth, minSize, depth+1,num_classes_unique)
            
        

In [284]:
#effettua il primo passo dell'algo di generazione dell'albero, richiama ricorsivamente sui figli
# VERSIONE CHE NON RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 
def startAlgo():
    dfForDTree=computeSubSeqDistance(df,CandidatesList)
    num_classes_unique=np.unique(num_classes)
    
    #inizio algo per nodo radice
    indexChosenAttribute,attributeValue,Dleft,Dright=findBestAttributeValue(dfForDTree,0)
    root=Node(indexChosenAttribute)
    numPattern=len(dfForDTree)
    entropy=computeEntropy(dfForDTree, num_classes_unique)
        
    #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
    nodeInfo=list()
    nodeInfo.append(attributeValue)
    nodeInfo.append(numPattern)
    nodeInfo.append(entropy)
    root.data=nodeInfo
    
    root.left=Node(indexChosenAttribute)
    root.right=Node(indexChosenAttribute)
    
    #chiamata ricorsiva
    if(len(Dleft)>0):
        buildTree(root.left,Dleft,3, 8,1,num_classes_unique)
    if(len(Dright)>0):
        buildTree(root.right,Dright,3, 8, 1,num_classes_unique)
    return root

In [285]:
#stampa dell'albero
def printAll(Root):
    if(Root.left==None and Root.right==None):
        print('foglia')
    print('Nodo: '+str(Root.value))
    df=Root.data
    print(df)
    print("\n")
    if(Root.left!=None):
        printAll(Root.left)
    if(Root.right!=None):
        printAll(Root.right)

In [302]:
albero=None
albero=startAlgo()
print(albero)
printAll(albero)
#FOGLIA CONTIENE :  class value/ num pattern / entorpy
#NODO CONTIENE:     value/ num pattern / entropy

55
55
60
115
MUTUALDF
55
0
16
True
1
29
True
2
38
False
BEST
38
55
55
60
115
MUTUALDF
55
0
16
True
1
15
False
BEST
15
profondità: 1
len dataset: 25
valore: 0.6166835003694846
scendo a sx splittando su: 15
55
55
60
115
MUTUALDF
55
0
31
False
BEST
31
profondità: 2
len dataset: 10
valore: 0.8771487609286801
scendo a sx splittando su: 31
scendo a dx splittando su: 31
scendo a dx splittando su: 15
55
55
60
115
MUTUALDF
55
0
13
False
BEST
13
profondità: 2
len dataset: 15
valore: 0.6595180364952272
scendo a sx splittando su: 13
scendo a dx splittando su: 13

           __________38
          /            \
     ____15___          -1
    /         \
  _31         _13
 /   \       /   \
-1    -1    -1    -1

Nodo: 38
[0.9466104526966131, 30, 1.5655962303576019]


Nodo: 15
[0.6166835003694846, 25, 1.516623811079363]


Nodo: 31
[0.8771487609286801, 10, 1.3709505944546687]


foglia
Nodo: -1
[0.0, 6, 0.9182958340544896]


foglia
Nodo: -1
[1.0, 4, 1.0]


Nodo: 13
[0.6595180364952272, 15, 1.158938532

In [ ]:
#INDEX CHOSEN ATTRIBUTE VA SOMMATO/SOTTRATTO AL NUMERO DI ATTRIBUTI CHE PASSO/NON PASSO A MUTUAL 
#FORSE RISOLVO EASY 

In [274]:
dfForDTree=computeSubSeqDistance(df,CandidateList)
y = dfForDTree['class'].values
y=y.astype('int')
numAttributes=len(dfForDTree.columns.values)
numAttributes-=2 #tolgo i due attributi TsIndex e class dal Dframe
vecMutualInfo=mutual_info_classif(dfForDTree.iloc[:,np.r_[:numAttributes]], y, discrete_features=False)
indice=getBestIndexAttribute(vecMutualInfo,2)
print(indice)
print(CandidatesUsedList.iloc[indice]['Used'])



115
55
60
115
candidatesIndex    110.000000
gain                 0.303862
Name: 110, dtype: float64
0
True
